In [19]:
import pickle 
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import torch.optim as optim

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

In [ ]:
file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\finalnew.txt', 'rb')
data = pickle.load(file)
#seperate out classes from inputs
classes, inputs = zip(*data)

#removing nan from inputs and convert to float
inputs_df = pd.DataFrame(inputs)
inputs_df.fillna(value=-1,inplace = True)
inputs = inputs_df.values.tolist()
inputs = [[float(i) for i in j] for j in inputs]

In [ ]:
REBUILD_DATA = False

class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\finalnew.txt', 'rb')
    data = pickle.load(file)
    #seperate out classes from inputs
    classes, inputs = zip(*data)
    
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(inputs)):
            if len(classes[i]) == 8:
                self.training_data.append([np.array(inputs[i]),np.array(classes[i])])
            else:
                excluded += 1
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)
        
if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(409, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x #nn.Softmax(x, dim=1)
        
        
net = Net().to(device)
print(net)    
#define tensors

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
    
Y_w = torch.tensor([i for i in Y_w])
    
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y_w[:-val_size]

test_X = X[-val_size:]
test_y = Y_w[-val_size:]
print(len(train_X), len(test_X))

test_X = test_X.to(device)
test_y = test_y.to(device)
    
def train(net):
    BATCH_SIZE = 100
    EPOCHS = 1000
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.05)

    for epoch in tqdm.tqdm(range(EPOCHS)):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            
            m = nn.LogSoftmax(dim = 1)
            outputs = m(net(batch_X))
            loss = loss_function(outputs, batch_y.float())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%100 == 0):
            print(f"Epoch: {epoch}. Loss: {loss}")
            test(net)
            
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm.tqdm(range(len(test_X))):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i]).to(device)  # returns a list, 
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

In [ ]:
X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])

In [ ]:
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)

In [ ]:
#Generates places
Y_p = []
for i in Y:
    place = [x*x for x in i]
    Y_p.append(place)
    
Y_p[1]

In [ ]:
Y[0]

In [ ]:
Y_w = torch.tensor([i for i in Y_w])
Y_w

In [ ]:
Y_w[1:10]

In [23]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(409, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x #nn.Softmax(x, dim=1)
        
        
net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=409, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=8, bias=True)
)


In [ ]:
import torch.optim as optim

In [ ]:
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y_w[:-val_size]

test_X = X[-val_size:]
test_y = Y_w[-val_size:]
print(len(train_X), len(test_X))

test_X = test_X.to(device)
test_y = test_y.to(device)

In [ ]:
#DONT RUN


VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y[:-val_size]

test_X = X[-val_size:]
test_y = Y[-val_size:]
print(len(train_X), len(test_X))



In [24]:
net = Net().to(device)

In [25]:
def train(net):
    BATCH_SIZE = 100
    EPOCHS = 10
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.05)

    for epoch in range(EPOCHS):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            #print(batch_X, batch_y)

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            optimizer.zero_grad()
            m = nn.LogSoftmax(dim = 1)
            outputs = m(net(batch_X))
            loss = loss_function(outputs, batch_y.float())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%100 != 0):
            print(f"Epoch: {epoch}. Loss: {loss}")
            test(net)

In [ ]:
train(net)

In [ ]:
def test(net):
    correct = 0
    total = 0
    realclasslist = []
    predictedClassList = []
    with torch.no_grad():
        for i in range(len(test_X)):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            realclasslist.append(test_y[i])
            net_out = net(test_X[i]).to(device)  # returns a list, 
            predictedClassList.append(net_out.tolist())
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))
    return (predictedClassList, realclasslist)

In [26]:
(predlist, realist) = test(net)

Epoch: 1. Loss: 5.074493885040283
Accuracy:  0.124
Epoch: 2. Loss: 5.066226482391357
Accuracy:  0.124
Epoch: 3. Loss: 5.057588577270508
Accuracy:  0.11
Epoch: 4. Loss: 5.049118995666504
Accuracy:  0.11
Epoch: 5. Loss: 5.041101932525635
Accuracy:  0.11
Epoch: 6. Loss: 5.033647060394287
Accuracy:  0.11
Epoch: 7. Loss: 5.026751518249512
Accuracy:  0.11
Epoch: 8. Loss: 5.020357608795166
Accuracy:  0.11
Epoch: 9. Loss: 5.014395236968994
Accuracy:  0.11


In [21]:
for i in range(0,1000):
    print(predlist[i])

[0.1536491960287094, 0.17514923214912415, 0.1201455146074295, 0.11960526555776596, -0.07425230741500854, 0.11345954239368439, 0.11437772959470749, 0.11158479005098343]
[0.1536491960287094, 0.17514923214912415, 0.1201455146074295, 0.11960526555776596, -0.07425230741500854, 0.11345954239368439, 0.11437772959470749, 0.11158479005098343]
[0.1536491960287094, 0.17514923214912415, 0.1201455146074295, 0.11960526555776596, -0.07425230741500854, 0.11345954239368439, 0.11437772959470749, 0.11158479005098343]
[0.1536491960287094, 0.17514923214912415, 0.1201455146074295, 0.11960526555776596, -0.07425230741500854, 0.11345954239368439, 0.11437772959470749, 0.11158479005098343]
[0.1536491960287094, 0.17514923214912415, 0.1201455146074295, 0.11960526555776596, -0.07425230741500854, 0.11345954239368439, 0.11437772959470749, 0.11158479005098343]
[0.1536491960287094, 0.17514923214912415, 0.1201455146074295, 0.11960526555776596, -0.07425230741500854, 0.11345954239368439, 0.11437772959470749, 0.11158479005

In [ ]:
resultdf = pd.DataFrame(predlist, columns = ['1', '2', '3', '4', '5', '6', '7' , '8'])